In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import cvxopt
import pulp
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ

In [2]:
# Ejemplo del problema de transporte de las cervezas utilizando Pyomo
# Creación del modelo
modelo = ConcreteModel()

# Planteamiento de los nodos de oferta y demanda
modelo.i = Set(initialize=["Planta 1","Planta 2", "Planta 3"], doc='Plantas')
modelo.j = Set(initialize=['Ciudad 1', 'Ciudad 2', 'Ciudad 3', 'Ciudad 4'],
               doc='Ciudades')

# Definicion de la capacidad de oferta y demanda
modelo.a = Param(modelo.i, initialize={'Planta 1':35,'Planta 2':50,'Planta 3':40},
                 doc='Capacidad de oferta de las plantas')
modelo.b = Param(modelo.j, initialize={'Ciudad 1':45,'Ciudad 2':20,'Ciudad 3':30, 
                                      'Ciudad 4':30},
                 doc='Demanda de cada ciudad')

# Costo del KWh x ciudad
costos = {
    ('Planta 1', 'Ciudad 1'): 8,
    ('Planta 1', 'Ciudad 2'): 6,
    ('Planta 1', 'Ciudad 3'): 10,
    ('Planta 1', 'Ciudad 4'): 9,
    ('Planta 2', 'Ciudad 1'): 9,
    ('Planta 2', 'Ciudad 2'): 12,
    ('Planta 2', 'Ciudad 3'): 13,
    ('Planta 2', 'Ciudad 4'): 7,
    ('Planta 3', 'Ciudad 1'): 14,
    ('Planta 3', 'Ciudad 2'): 9,
    ('Planta 3', 'Ciudad 3'): 16,
    ('Planta 3', 'Ciudad 4'): 5,
    }
modelo.d = Param(modelo.i, modelo.j, initialize=costos, 
                doc='Costo de enviar energía')

# Coste de transporte
def f_costo(modelo, i, j):
    return modelo.d[i,j]
modelo.c = Param(modelo.i, modelo.j, initialize=f_costo, 
                doc='Costo de transporte')


modelo.x = Var(modelo.i, modelo.j, bounds=(0.0,None), 
              doc='Cantidad de millones (KWh)')

In [3]:
## Definimos las restricciones ##
# Límite de oferta
def f_oferta(modelo, i):
    return sum(modelo.x[i,j] for j in modelo.j) <= modelo.a[i]
modelo.oferta = Constraint(modelo.i, rule=f_oferta, 
                           doc='Límites oferta de cada Planta')

# Límite de demanda
def f_demanda(modelo, j):
    return sum(modelo.x[i,j] for i in modelo.i) >= modelo.b[j]  
modelo.demanda = Constraint(modelo.j, rule=f_demanda, 
                            doc='Límites demanda de cada Ciudad')

In [4]:
## Definimos la función objetivo y resolvemos el problema ##
# Función objetivo
def f_objetivo(modelo):
    return sum(modelo.c[i,j]*modelo.x[i,j] for i in modelo.i for j in modelo.j)
modelo.objetivo = Objective(rule=f_objetivo, sense=minimize, 
                           doc='Función Objetivo')

# resolvemos el problema e imprimimos resultados
def pyomo_postprocess(options=None, instance=None, results=None):
    modelo.x.display()

# utilizamos solver glpk
opt = SolverFactory("glpk")
resultados = opt.solve(modelo)

# imprimimos resultados
print("\nSolución óptima encontrada\n" + '-'*80)
pyomo_postprocess(None, None, resultados)


Solución óptima encontrada
--------------------------------------------------------------------------------
x : Cantidad de millones (KWh)
    Size=12, Index=x_index
    Key                      : Lower : Value : Upper : Fixed : Stale : Domain
    ('Planta 1', 'Ciudad 1') :   0.0 :   0.0 :  None : False : False :  Reals
    ('Planta 1', 'Ciudad 2') :   0.0 :  10.0 :  None : False : False :  Reals
    ('Planta 1', 'Ciudad 3') :   0.0 :  25.0 :  None : False : False :  Reals
    ('Planta 1', 'Ciudad 4') :   0.0 :   0.0 :  None : False : False :  Reals
    ('Planta 2', 'Ciudad 1') :   0.0 :  45.0 :  None : False : False :  Reals
    ('Planta 2', 'Ciudad 2') :   0.0 :   0.0 :  None : False : False :  Reals
    ('Planta 2', 'Ciudad 3') :   0.0 :   5.0 :  None : False : False :  Reals
    ('Planta 2', 'Ciudad 4') :   0.0 :   0.0 :  None : False : False :  Reals
    ('Planta 3', 'Ciudad 1') :   0.0 :   0.0 :  None : False : False :  Reals
    ('Planta 3', 'Ciudad 2') :   0.0 :  10.0 :  None 